# SMDP Progetto mappe

## Dipendenze

In [ ]:
# =========================
# Dipendenze (minime)
# =========================
import os
import math
import numpy as np
import pandas as pd
import geopandas as gpd
import networkx as nx
import osmnx as ox
import matplotlib.pyplot as plt
from shapely.geometry import Point
from matplotlib.patches import Patch

# (opzionale) basemap
try:
    import contextily as ctx
    HAS_CTX = True
except Exception:
    HAS_CTX = False

## PARAMETRI 

In [3]:

# =========================
# PARAMETRI
# =========================
center_latlon = (41.940583, 12.418912)   # (lat, lon) Santa Maria della Pietà
dist = 2000                               # raggio (metri)
crs_metric = "EPSG:32633"                 # UTM 33N per Roma

# =========================
# 1) GRAFO PEDONALE OSM + PROIEZIONE
# =========================
G_ll = ox.graph_from_point(center_latlon, dist=dist, network_type="walk", simplify=True)
center_node = ox.distance.nearest_nodes(G_ll, X=center_latlon[1], Y=center_latlon[0])
G = ox.project_graph(G_ll, to_crs=crs_metric)

# =========================
# 2) DISTANZE DI RETE DAL CENTRO (metri)
# =========================
distances = nx.single_source_dijkstra_path_length(G, center_node, weight="length")
nx.set_node_attributes(G, values=distances, name="dist_to_center_m")
for u, v, k, data in G.edges(keys=True, data=True):
    du = distances.get(u, np.nan)
    dv = distances.get(v, np.nan)
    data["dist_edge_m"] = np.nanmin([du, dv])

# =========================
# 3) GDFS NODI/ARCHI + PUNTO CENTRO
# =========================
nodes_gdf, edges_gdf = ox.graph_to_gdfs(G, nodes=True, edges=True)
smdp_point = gpd.GeoDataFrame(
    {"name": ["Santa Maria della Pietà"]},
    geometry=[Point(center_latlon[1], center_latlon[0])],
    crs="EPSG:4326"
).to_crs(crs_metric)

# =========================
# 4) POI OSM
# =========================
poi_tags = {
    "amenity": [
        "school","university","library","hospital",
        "clinic","doctors","pharmacy",
        "park","cinema","theatre","museum","arts_centre",
        "place_of_worship","community_centre",
        "sports_centre","swimming_pool","gym","stadium","pitch"
    ]
}
poi_gdf = ox.features_from_point(center_latlon, tags=poi_tags, dist=dist)
poi_gdf = poi_gdf[poi_gdf.geometry.notna()].to_crs(crs_metric).copy()
# (Niente representative_point: conserviamo le forme originali)

# =========================
# 5) CATEGORIZZAZIONE + COLORI (forme inalterate)
# =========================
def amenity_to_category(a):
    if a in {"hospital","clinic","doctors","pharmacy"}:
        return "salute"
    if a in {"school","university","college","kindergarten","library"}:
        return "istruzione"
    if a in {"museum","theatre","cinema","arts_centre"}:
        return "cultura"
    if a in {"sports_centre","swimming_pool","gym","stadium","pitch"}:
        return "sport"
    if a in {"place_of_worship"}:
        return "religione"
    if a in {"park","community_centre"}:
        return "spazio_pubblico"
    return "altro"

cat_order = ["salute","istruzione","cultura","sport","religione","spazio_pubblico"]
color_map = {
    "salute": "#e41a1c",
    "istruzione": "#377eb8",
    "cultura": "#4daf4a",
    "sport": "#984ea3",
    "religione": "#ff7f00",
    "spazio_pubblico": "#a6cee3",
}

poi_gdf["category"] = poi_gdf["amenity"].map(amenity_to_category)
poi_cat = poi_gdf[poi_gdf["category"].isin(cat_order)].copy()
counts = poi_cat["category"].value_counts().to_dict()

def plot_poi_colored(ax):
    """Plotta i POI per categoria mantenendo le forme originali (punti/poligoni)."""
    for cat in cat_order:
        sub = poi_cat[poi_cat["category"] == cat]
        if sub.empty:
            continue
        sub.plot(
            ax=ax,
            color=color_map.get(cat, "#7f7f7f"),  # per poligoni = facecolor; per punti = marker color
            edgecolor="white",
            linewidth=0.4,
            markersize=28,       # usato solo dai punti; ignorato dai poligoni
            alpha=0.9,
            zorder=4
        )



In [ ]:
from shapely.geometry import Polygon
from shapely.affinity import translate
import math
nodes_gdf, edges_gdf = ox.graph_to_gdfs(G, nodes=True, edges=True)

# ===== HEX GRID sul poligono di studio =====
def make_hex_grid(polygon, hex_size_m, crs):
    """
    Crea una griglia di esagoni regolari (lato ~ hex_size_m) clip-pata sul poligono.
    """
    poly = gpd.GeoSeries([polygon], crs=crs)
    minx, miny, maxx, maxy = poly.total_bounds
    dx = hex_size_m * 3**0.5   # passo in x
    dy = hex_size_m * 1.5      # passo in y

    hexes = []
    y = miny - dy
    row = 0
    while y < maxy + dy:
        x = minx - dx
        # offset a righe alterne
        if row % 2 == 1:
            x += dx / 2.0
        while x < maxx + dx:
            # costruisci esagono centrato in (x,y)
            pts = []
            for k in range(6):
                angle = math.radians(60 * k + 30)  # “pointy-top”
                px = x + hex_size_m * math.cos(angle)
                py = y + hex_size_m * math.sin(angle)
                pts.append((px, py))
            hexes.append(Polygon(pts))
            x += dx
        y += dy
        row += 1

    grid = gpd.GeoDataFrame(geometry=hexes, crs=crs)
    grid = gpd.overlay(grid, gpd.GeoDataFrame(geometry=[polygon], crs=crs), how="intersection")
    grid["hex_area_km2"] = grid.geometry.area / 1e6
    return grid

# ===== Densità proxy dai buildings OSM =====
def get_building_density_hex(edges_gdf, hex_size_m=300):
    """
    1) definisce l’area di studio (buffer attorno alla rete);
    2) scarica edifici OSM in quell’area;
    3) calcola superficie edificata per esagono e la converte in densità (km²).
    Ritorna (grid_hex, buildings_gdf).
    """
    # area di studio: buffer attorno alle strade
    study_poly = edges_gdf.unary_union.buffer(200)  # 200 m margine
    # scarica edifici dentro l’area (serve in WGS84)
    study_poly_wgs = gpd.GeoSeries([study_poly], crs=crs_metric).to_crs("EPSG:4326").iloc[0]
    bld = ox.features_from_polygon(study_poly_wgs, tags={"building": True})
    bld = bld[bld.geometry.notna()].copy()
    bld = bld.to_crs(crs_metric)

    # tieni solo poligoni/multipoligoni
    bld = bld[bld.geometry.geom_type.isin(["Polygon", "MultiPolygon"])].copy()
    bld["bld_area_m2"] = bld.geometry.area

    # griglia esagonale
    grid = make_hex_grid(study_poly, hex_size_m, crs_metric)

    # intersect (clip) e somma area edificata per esagono
    inter = gpd.overlay(bld[["bld_area_m2", "geometry"]], grid[["geometry"]], how="intersection")
    inter["part_area_m2"] = inter.geometry.area
    # peso: se l’edificio viene tagliato dall’esagono, usa l’area della parte
    # (alternativa: proporzione * bld_area_m2, molto simile in pratica)
    s = inter.groupby(inter.index_right)["part_area_m2"].sum()
    grid["bld_area_m2"] = grid.index.map(s).fillna(0.0)

    # densità proxy: m² edificati per km²
    grid["bld_area_km2"] = grid["bld_area_m2"] / 1e6
    grid["bld_density_proxy"] = grid["bld_area_km2"] / grid["hex_area_km2"].replace(0, np.nan)
    return grid, bld

# ===== Quartieri da OSM =====
def get_quartieri_roma(crs_target):
    """
    Boundary dei quartieri (place=suburb/neighbourhood/quarter) per Roma.
    Filtra anche per 'Quartiere' / 'Q.' nel nome, se presente.
    """
    try:
        q = ox.features_from_place(
            "Roma, Lazio, Italia",
            tags={"place": ["suburb", "neighbourhood", "quarter"]}
        )
        q = q[q.geometry.notna()].copy()
        namecol = next((c for c in ["name","official_name","short_name"] if c in q.columns), None)
        if namecol:
            mask = q[namecol].astype(str).str.contains(r"(Quartiere|Q\.)", case=False, na=False)
            # se il filtro è troppo restrittivo, tieni tutto
            if mask.sum() >= 5:
                q = q[mask]
        q = q.to_crs(crs_target)
        q["label_pt"] = q.geometry.representative_point()
        return q
    except Exception:
        return gpd.GeoDataFrame(geometry=[], crs=crs_target)

# =========================
# 6) PLOT — HEX DENSITY (proxy) + STRADE per distanza + POI + QUARTIERI
# =========================

# 0) calcola griglia esagonale di densità (solo OSM buildings)
hex_grid, bld_gdf = get_building_density_hex(edges_gdf, hex_size_m=300)

# 1) quartieri
quartieri = get_quartieri_roma(crs_metric)

fig, ax = plt.subplots(figsize=(11.5, 9.5))

# A) Densità proxy (sotto tutto)
if not hex_grid.empty:
    hex_grid.plot(
        ax=ax, column="bld_density_proxy",
        cmap="OrRd", alpha=0.45, edgecolor="none",
        legend=True, legend_kwds={"label": "Densità proxy edificato (m² edificati per km²)", "shrink": 0.6}
    )

# B) Basemap
try:
    ctx.add_basemap(ax, crs=crs_metric, source=ctx.providers.CartoDB.Positron)
except Exception:
    pass

# C) Strade colorate per distanza
edges_plot = edges_gdf.sort_values("dist_edge_m")
edges_plot.plot(
    ax=ax, column="dist_edge_m", cmap="viridis_r",
    linewidth=1.2, legend=True,
    legend_kwds={"label": "Distanza di rete dal centro (m)", "shrink": 0.6}
)

# D) POI (forme originali, solo colore)
if not poi_cat.empty:
    plot_poi_colored(ax)

# E) Confini quartieri + etichette
if not quartieri.empty:
    quartieri.boundary.plot(ax=ax, color="dimgray", linewidth=0.8, alpha=0.9, zorder=6)
    namecol = next((c for c in ["name","official_name","short_name"] if c in quartieri.columns), None)
    if namecol:
        for _, r in quartieri.iterrows():
            pt = r["label_pt"]
            ax.text(pt.x, pt.y, s=str(r[namecol]),
                    fontsize=7.5, weight="bold", color="black",
                    ha="center", va="center", zorder=7)

# F) Centro (stella)
smdp_point.plot(ax=ax, color='gold', edgecolor='black', markersize=180, marker='*', zorder=8)

# (opzionale) KNN di rete se già creato
try:
    if "knn_net_edges" in globals() and isinstance(knn_net_edges, gpd.GeoDataFrame) and not knn_net_edges.empty:
        knn_net_edges.to_crs(crs_metric).plot(ax=ax, color='white', alpha=0.4, linewidth=1.0, zorder=5)
except Exception:
    pass

# Titolo & stile
ax.set_title("Walkability verso Santa Maria della Pietà\nDistanza di rete + densità edilizia (proxy OSM) + Quartieri",
             fontsize=14, fontweight="bold")
ax.set_aspect("equal"); ax.axis("off")

# Legenda categorie amenity
legend_handles = [
    Patch(facecolor=color_map[c], edgecolor='white', linewidth=0.6,
          label=f"{c} ({counts.get(c,0)})")
    for c in cat_order if c in counts
]
fig.legend(handles=legend_handles, loc="center left",
           bbox_to_anchor=(1.02, 0.5),
           frameon=True, title="Categorie amenity")

plt.tight_layout()
plt.show()


In [4]:
# =========================
# Dipendenze (minime)
# =========================
import os
import math
import numpy as np
import pandas as pd
import geopandas as gpd
import networkx as nx
import osmnx as ox
import matplotlib.pyplot as plt
from shapely.geometry import Point
from matplotlib.patches import Patch

# (opzionale) basemap
try:
    import contextily as ctx
    HAS_CTX = True
except Exception:
    HAS_CTX = False

# =========================
# 0) PARAMETRI GLOBALI
# =========================
PARAMS = {
    "center_latlon": (41.940583, 12.418912),  # S. Maria della Pietà
    "dist_m": 2000,
    "crs_metric": "EPSG:32633",               # UTM 33N Roma
    "network_type": "walk",
    "basemap": True,                          # basemap CartoDB Positron
    "draw_quartieri": True,                   # confini + label quartieri
    "amenity_tags": {
        "amenity": [
            "school","university","library","hospital",
            "clinic","doctors","pharmacy",
            "park","cinema","theatre","museum","arts_centre",
            "place_of_worship","community_centre",
            "sports_centre","swimming_pool","gym","stadium","pitch"
        ]
    },
}

# palette e mapping categorie
CAT_ORDER = ["salute","istruzione","cultura","sport","religione","spazio_pubblico"]
COLOR_MAP = {
    "salute": "#e41a1c",
    "istruzione": "#377eb8",
    "cultura": "#4daf4a",
    "sport": "#984ea3",
    "religione": "#ff7f00",
    "spazio_pubblico": "#a6cee3",
}
def amenity_to_category(a: str) -> str:
    if a in {"hospital","clinic","doctors","pharmacy"}:
        return "salute"
    if a in {"school","university","college","kindergarten","library"}:
        return "istruzione"
    if a in {"museum","theatre","cinema","arts_centre"}:
        return "cultura"
    if a in {"sports_centre","swimming_pool","gym","stadium","pitch"}:
        return "sport"
    if a in {"place_of_worship"}:
        return "religione"
    if a in {"park","community_centre"}:
        return "spazio_pubblico"
    return "altro"

# =========================
# 1) IMPOSTAZIONI OSMNX
# =========================
ox.settings.use_cache = True     # cache = più veloce dopo il primo run
ox.settings.log_console = False
ox.settings.timeout = 180


# =========================
# 2) FUNZIONI MODULARI
# =========================
def build_walk_graph(center_latlon, dist_m, crs_metric, network_type="walk"):
    """Scarica rete pedonale intorno al centro e proietta in CRS metrico."""
    G_ll = ox.graph_from_point(center_latlon, dist=dist_m, network_type=network_type, simplify=True)
    center_node_ll = ox.distance.nearest_nodes(G_ll, X=center_latlon[1], Y=center_latlon[0])
    # proietta il grafo (nodi e archi cambiano id, ricalcoliamo il nodo più vicino dopo)
    G = ox.project_graph(G_ll, to_crs=crs_metric)
    # trova nodo più vicino al centro nella proiezione
    center_xy = gpd.GeoSeries([Point(center_latlon[1], center_latlon[0])], crs="EPSG:4326").to_crs(crs_metric).iloc[0]
    # estrai nodo più vicino in proiezione
    nodes_gdf, _ = ox.graph_to_gdfs(G, nodes=True, edges=False)
    center_node = nodes_gdf.geometry.distance(Point(center_xy.x, center_xy.y)).sort_values().index[0]
    return G, center_node


def compute_network_distances(G, center_node, weight="length"):
    """Distanze di rete (Dijkstra) dal nodo centrale. Aggiunge anche dist_edge_m agli archi."""
    # path length in metri
    distances = nx.single_source_dijkstra_path_length(G, center_node, weight=weight)
    nx.set_node_attributes(G, values=distances, name="dist_to_center_m")
    # per ogni arco, min dei due estremi
    for u, v, k, data in G.edges(keys=True, data=True):
        du = distances.get(u, np.nan)
        dv = distances.get(v, np.nan)
        data["dist_edge_m"] = np.nanmin([du, dv])
    return G


def graph_to_gdfs(G):
    """Converte grafo in GeoDataFrame nodi/archi."""
    nodes_gdf, edges_gdf = ox.graph_to_gdfs(G, nodes=True, edges=True)
    return nodes_gdf, edges_gdf


def fetch_pois(center_latlon, tags, crs_metric, dist_m):
    """Scarica POI OSM (forme originali) nel raggio e li proietta in CRS metrico."""
    poi = ox.features_from_point(center_latlon, tags=tags, dist=dist_m)
    poi = poi[poi.geometry.notna()].copy()
    poi = poi.to_crs(crs_metric)
    # categorizzazione
    if "amenity" in poi.columns:
        poi["category"] = poi["amenity"].map(amenity_to_category)
    else:
        poi["category"] = "altro"
    poi = poi[poi["category"].isin(CAT_ORDER)]
    return poi


def fetch_quartieri_roma(crs_target):
    """Scarica quartieri (suburb/neighbourhood/quarter) e crea label point."""
    try:
        q = ox.features_from_place(
            "Roma, Lazio, Italia",
            tags={"place": ["suburb", "neighbourhood", "quarter"]}
        )
        q = q[q.geometry.notna()].copy()
        namecol = next((c for c in ["name","official_name","short_name"] if c in q.columns), None)
        if namecol:
            mask = q[namecol].astype(str).str.contains(r"(Quartiere|Q\.)", case=False, na=False)
            if mask.sum() >= 5:
                q = q[mask]
        q = q.to_crs(crs_target)
        q["label_pt"] = q.geometry.representative_point()
        return q
    except Exception:
        return gpd.GeoDataFrame(geometry=[], crs=crs_target)


def plot_poi_by_category(ax, poi_gdf):
    """Plotta POI rispettando forme (punti/poligoni), solo colore per categoria."""
    if poi_gdf.empty:
        return {}, []
    counts = poi_gdf["category"].value_counts().to_dict()
    for cat in CAT_ORDER:
        sub = poi_gdf[poi_gdf["category"] == cat]
        if sub.empty:
            continue
        sub.plot(
            ax=ax,
            color=COLOR_MAP.get(cat, "#7f7f7f"),  # colora facce poligoni o marker dei punti
            edgecolor="white",
            linewidth=0.4,
            markersize=28,
            alpha=0.9,
            zorder=4
        )
    # handle legenda custom
    handles = [
        Patch(facecolor=COLOR_MAP[c], edgecolor='white', linewidth=0.6,
              label=f"{c} ({counts.get(c,0)})")
        for c in CAT_ORDER if counts.get(c,0) > 0
    ]
    return counts, handles


def make_map(PARAMS):
    """Pipeline leggera: rete per distanza + POI + (opzionale) quartieri + (opzionale) basemap."""
    center_latlon = PARAMS["center_latlon"]
    dist_m       = PARAMS["dist_m"]
    crs_metric   = PARAMS["crs_metric"]

    # 1) Rete pedonale
    G, center_node = build_walk_graph(center_latlon, dist_m, crs_metric, network_type=PARAMS["network_type"])

    # 2) Distanze di rete
    G = compute_network_distances(G, center_node, weight="length")

    # 3) GDFs
    nodes_gdf, edges_gdf = graph_to_gdfs(G)

    # 4) POI
    poi_gdf = fetch_pois(center_latlon, PARAMS["amenity_tags"], crs_metric, dist_m)

    # 5) Quartieri (opzionale)
    quartieri = fetch_quartieri_roma(crs_metric) if PARAMS["draw_quartieri"] else gpd.GeoDataFrame(geometry=[], crs=crs_metric)

    # 6) Centro (stella)
    smdp_point = gpd.GeoDataFrame(
        {"name": ["Santa Maria della Pietà"]},
        geometry=[Point(center_latlon[1], center_latlon[0])],
        crs="EPSG:4326"
    ).to_crs(crs_metric)

    # 7) PLOT
    fig, ax = plt.subplots(figsize=(11.5, 9.5))

    # A) Basemap (sotto tutto)
    if PARAMS["basemap"] and HAS_CTX:
        try:
            # disegna prima qualcosa per definire extent
            edges_gdf.plot(ax=ax, color="none")
            ctx.add_basemap(ax, crs=crs_metric, source=ctx.providers.CartoDB.Positron)
        except Exception:
            pass

    # B) Strade per distanza (layer principale)
    edges_plot = edges_gdf.sort_values("dist_edge_m")
    edges_plot.plot(
        ax=ax, column="dist_edge_m", cmap="viridis_r",
        linewidth=1.2, legend=True,
        legend_kwds={"label": "Distanza di rete dal centro (m)", "shrink": 0.6}
    )

    # C) POI per categoria (forme originali)
    counts, handles = plot_poi_by_category(ax, poi_gdf)

    # D) Quartieri (bordo + label)
    if not quartieri.empty:
        quartieri.boundary.plot(ax=ax, color="dimgray", linewidth=0.8, alpha=0.9, zorder=6)
        namecol = next((c for c in ["name","official_name","short_name"] if c in quartieri.columns), None)
        if namecol:
            for _, r in quartieri.iterrows():
                pt = r["label_pt"]
                ax.text(pt.x, pt.y, s=str(r[namecol]),
                        fontsize=7.5, weight="bold", color="black",
                        ha="center", va="center", zorder=7)

    # E) Centro (stella)
    smdp_point.plot(ax=ax, color='gold', edgecolor='black', markersize=180, marker='*', zorder=8)

    # Stile
    ax.set_title("Walkability verso Santa Maria della Pietà\nDistanza di rete + POI + (Quartieri opzionali)",
                 fontsize=14, fontweight="bold")
    ax.set_aspect("equal"); ax.axis("off")

    # Legenda categorie amenity
    if handles:
        fig.legend(handles=handles, loc="center left",
                   bbox_to_anchor=(1.02, 0.5),
                   frameon=True, title="Categorie amenity")

    plt.tight_layout()
    plt.show()

    return {
        "G": G,
        "nodes_gdf": nodes_gdf,
        "edges_gdf": edges_gdf,
        "poi_gdf": poi_gdf,
        "quartieri": quartieri,
        "smdp_point": smdp_point
    }


# =========================
# 3) ESECUZIONE
# =========================
if __name__ == "__main__":
    outputs = make_map(PARAMS)


ValueError: too many values to unpack (expected 2)